In [4]:
import tensorflow as tf
import numpy as np

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()


# adding another channel -> grayscale to rgb conversion
train_images = np.stack([train_images]*3, axis=-1)
test_images = np.stack([test_images]*3, axis=-1)

# data normalization
train_images = train_images / 255.0
test_images = test_images / 255.0

# mnist data comes with 28x28 dimension most model like VGG, ResNet only takes 32x32 dimensioned data
train_images = tf.image.resize(train_images, [32, 32])
test_images = tf.image.resize(test_images, [32, 32])


# one hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
from tensorflow.keras.applications import MobileNetV2
base_model = MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights='imagenet')
base_model.trainable = False
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import  Model

inputs = Input(shape=(32, 32, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
# x = Dropout(0.2)(x)
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy',
    metrics=['accuracy'])

/tmp/ipython-input-1936179883.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=5, batch_size=32)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9667 - loss: 0.1309 - val_accuracy: 0.9643 - val_loss: 0.1673
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9727 - loss: 0.1067 - val_accuracy: 0.9608 - val_loss: 0.1664
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9751 - loss: 0.0947 - val_accuracy: 0.9635 - val_loss: 0.1481
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9784 - loss: 0.0717 - val_accuracy: 0.9638 - val_loss: 0.1521
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9790 - loss: 0.0770 - val_accuracy: 0.9688 - val_loss: 0.1187


In [11]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy * 100:.2f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9627 - loss: 0.1382
Test loss: 0.11868084967136383
Test accuracy: 96.88%


In [6]:
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False



In [7]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy',
    metrics=['accuracy'])
model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=5, batch_size=32)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 54s 15ms/step - accuracy: 0.8306 - loss: 0.6188 - val_accuracy: 0.7005 - val_loss: 2.3312
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9324 - loss: 0.2741 - val_accuracy: 0.6162 - val_loss: 7.9278
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9424 - loss: 0.2309 - val_accuracy: 0.9483 - val_loss: 0.2191
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9610 - loss: 0.1532 - val_accuracy: 0.9406 - val_loss: 0.3160
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9647 - loss: 0.1398 - val_accuracy: 0.9035 - val_loss: 0.4302


In [8]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy * 100:.2f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8995 - loss: 0.4687
Test loss: 0.43016692996025085
Test accuracy: 90.35%
